### This notebook helps to do 3 things:
- Paso 1: Excecute the docXChain to process a pdf into JSON (semi-parametrizado)
- Paso 2: Convert Json into DataFrame
- Pas 3: DataFrame to Document

In [16]:
#- Paso 1: Exe DocXChain
import subprocess

path_to_docXchain = "/home/camilo/Documents/01-tutoriales/docxchain/Applications/DocXchain/"
path_to_pdfs = "/home/camilo/Documents/by_species/"
species_folder = "Leptonycteris/"
arg1_fun = "whole_pdf_conversion"
arg2_input = "Gómez-Ruiz et al., 2015.pdf"
arg3_output = "Gómez-Ruiz et al., 2015.json"
#Hensley_test.json"

args = ["python", path_to_docXchain+"example.py", arg1_fun, path_to_pdfs+species_folder+arg2_input, path_to_docXchain+"output/"+arg3_output]
#args = ["python", "-c", f"import example; example.whole_pdf_conversion('{path_to_pdfs+species_folder+arg2_input}', '{path_to_docXchain+'output/'+arg3_output}')"]
result = subprocess.run(args, capture_output=True, text=True)

print(result.stdout)
print(result.stderr)

KeyboardInterrupt: 

### - Paso 2: Convert Json into csv

In [23]:
import pandas as pd
import json
file_name = path_to_docXchain+"output/"+arg3_output
#file_name = "test_no-image.json"

with open(file_name, 'r') as f:
    data = json.load(f)
print(data[0].keys())

dict_keys(['page', 'information'])


In [24]:
df = pd.DataFrame(data)
new_df = df
# you could try 
# col = df.map(type).astype(str).eq("<class 'list'>").all().idxmax()
# 'region_poly' would satisfy that condition but doesn't need to be expoloted. Besides, the structure should be stable
# Hence: column_names hard-code
cols = ['information','text_list']

for col in cols:
    print(col)
    new_df = new_df.explode(col).reset_index(drop=True)
    new_df = new_df.drop(columns=[col]).join(new_df[col].apply(pd.Series), rsuffix=f".{col}")

new_df

information
text_list


,page,category_index,category_name,region_poly,position,content,0
0,0,0,title,"[152, 303, 1162, 303, 1162, 431, 152, 431]","[215, 300, 1153, 303, 1153, 334, 215, 331]",[Conservacion de murcielagos nectarivoros],NaN
1,0,0,title,"[152, 303, 1162, 303, 1162, 431, 152, 431]","[157, 351, 1155, 348, 1155, 383, 157, 385]",[Phyllostomidae: Glossophagini) en riesgo en],NaN
2,0,0,title,"[152, 303, 1162, 303, 1162, 431, 152, 431]","[648, 401, 1149, 400, 1149, 434, 648, 434]",[CoahuilayNuevo Leon],NaN
3,0,2,plain text,"[207, 1125, 1099, 1125, 1099, 1235, 207, 1235]","[233, 1123, 1095, 1122, 1096, 1139, 233, 1141]",[Results: We report six caves used by nectar-f...,NaN
4,0,2,plain text,"[207, 1125, 1099, 1125, 1099, 1235, 207, 1235]","[204, 1146, 1094, 1147, 1094, 1163, 204, 1163]",[The community-based conservation strateqies t...,NaN
...,...,...,...,...,...,...,...
677,12,2,plain text,"[299, 1229, 539, 1229, 539, 1253, 299, 1253]","[296, 1230, 350, 1231, 350, 1251, 296, 1251]",[b) No],NaN
678,12,2,plain text,"[300, 387, 505, 387, 505, 415, 300, 415]","[296, 389, 376, 389, 376, 409, 296, 409]",[b). No sé],NaN
679,12,3,header,"[986, 58, 1135, 58, 1135, 76, 986, 76]","[985, 55, 1131, 55, 1131, 74, 985, 74]",[Gomez-Ruiz et al.],NaN
680,12,4,page number,"[1123, 1559, 1156, 1559, 1156, 1577, 1123, 1577]","[1107, 1557, 1151, 1557, 1151, 1576, 1107, 1575]",[101],NaN


In [25]:
file_outout_name = file_name.split(".")[0]+".csv"
print(file_outout_name)
new_df.to_csv(file_outout_name, header=True, index=False)

/home/camilo/Documents/01-tutoriales/docxchain/Applications/DocXchain/output/Gómez-Ruiz et al.csv


### - Paso 3: DataFrame to Document

In [120]:
# almost every error comes from not having the right data_type in column content. 
# It's very convenient to filter out problematic and non relevant categories from the beguinning
new_df.category_name.unique()
#new_df[new_df.category_name == 'figure']
#new_df[new_df.category_name == 'table']
#clean_df['content'] = new_df['content'].apply(lambda x: x[0] if isinstance(x,list) else '')
#clean_df['content']

array(['title', 'plain text', 'header', 'footnote', 'footer', 'figure',
       'page number', 'figure caption', 'table', 'table caption'],
      dtype=object)

In [32]:
df = pd.DataFrame(data)
new_df = df
# you could try 
# col = df.map(type).astype(str).eq("<class 'list'>").all().idxmax()
# 'region_poly' would satisfy that condition but doesn't need to be expoloted. Besides, the structure should be stable
# Hence: column_names hard-code
cols = ['information','text_list']

for col in cols:
    print(col)
    new_df = new_df.explode(col).reset_index(drop=True)
    new_df = new_df.drop(columns=[col]).join(new_df[col].apply(pd.Series), rsuffix=f".{col}")

new_df['content'] = new_df['content'].apply(lambda x: x[0] if isinstance(x,list) else '')
new_df['region_poly'] = new_df['region_poly'].apply(lambda x: tuple(x)) # lists are not mutable 
new_df['content_type'] = new_df['content'].apply(lambda x: type(x).__name__)

df_text = new_df[new_df.category_name == 'plain text']


information
text_list


In [33]:
aggr_level = ['page', 'region_poly']
grouped_df = df_text.groupby(aggr_level)

# Concatenate the content of the 'content' column with spaces
test = df_text.groupby(aggr_level)['content'].apply(lambda x: ' '.join(x)).reset_index()
test

,page,region_poly,content
0,0,"(143, 169, 472, 169, 472, 234, 143, 234)",Conservation nectar bats (Phyllostomidae:Gloss...
1,0,"(203, 1354, 1099, 1354, 1099, 1395, 203, 1395)","Key words: environmental education, conservati..."
2,0,"(205, 1252, 1100, 1252, 1100, 1338, 205, 1338)",Conclusions: We propose a conservation network...
3,0,"(206, 779, 1109, 779, 1109, 984, 206, 984)",Introduction: Cave-dwellinq nectar-feeding bat...
4,0,"(206, 997, 1099, 997, 1099, 1109, 206, 1109)",Material and methods: To document caves used b...
...,...,...,...
110,12,"(372, 602, 567, 602, 567, 624, 372, 624)",a): Control de plagas
111,12,"(374, 706, 580, 706, 580, 729, 374, 729)",d). Son depredadores
112,12,"(375, 741, 510, 741, 510, 763, 375, 763)",e). Otra: Cuál?
113,12,"(377, 636, 528, 636, 528, 658, 377, 658)",b). Polinizadores


In [35]:
from langchain_core.documents import Document
my_doc = Document(page_content=test)


<bound method DataFrame.to_markdown of      page                                     region_poly  \
0       0        (143, 169, 472, 169, 472, 234, 143, 234)   
1       0  (203, 1354, 1099, 1354, 1099, 1395, 203, 1395)   
2       0  (205, 1252, 1100, 1252, 1100, 1338, 205, 1338)   
3       0      (206, 779, 1109, 779, 1109, 984, 206, 984)   
4       0    (206, 997, 1099, 997, 1099, 1109, 206, 1109)   
..    ...                                             ...   
110    12        (372, 602, 567, 602, 567, 624, 372, 624)   
111    12        (374, 706, 580, 706, 580, 729, 374, 729)   
112    12        (375, 741, 510, 741, 510, 763, 375, 763)   
113    12        (377, 636, 528, 636, 528, 658, 377, 658)   
114    12        (379, 671, 617, 671, 617, 695, 379, 695)   

                                               content  
0    Conservation nectar bats (Phyllostomidae:Gloss...  
1    Key words: environmental education, conservati...  
2    Conclusions: We propose a conservation network...  


In [37]:
from langchain_core.documents import Document
from pymupdf4llm import to_markdown
from pathlib import Path

In [42]:
for row in test:
    my_doc = Document(row["content"])

TypeError: string indices must be integers

In [ ]:
metadata = dict(
                    source= Path(file_name).name,
                    species_folder =Path(file_name).parent.name,  
                    page = test["page"],
                    region_poly = test["region_poly"]
                )